# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

Assignment Collaborated with Tanmai Gajula(tanmai.gajula@colorado.edu)

In [5]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [6]:
from pyspark.sql.functions import col, count, countDistinct

We create spark session as seen in the tutorials

In [7]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()


The above statement reads the citation data which is actually not loaded untill it encounters an action

In [9]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [10]:
citations.show(5) # This is an Action which means that This is when the data is loaded and the first 5 rows are returned.

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows




The above statement reads the patents data which is actually not loaded untill it encounters an action

In [11]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [12]:
patents.show(5) # This is an Action which means that This is when the data is loaded and the first 5 rows are returned.

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

Select PATENT and POSTATE columns from patents dataframe and assign it to new dataframe: df [Transformation]

In [13]:
df = patents.select("PATENT", "POSTATE") 

we cache the pysaprk dataframe which is also a lazy operation - and is actually triggered when we encounter an action later.

In [14]:
df.cache()

DataFrame[PATENT: int, POSTATE: string]

Next, we Perform left join on citations pyspark dataframe and df with citations.CITING == df.PATENT, and drop PATENT column from result and rename POSTATE column to CITING_STATE

In [15]:
df1 = citations.join(df, citations.CITING == df.PATENT, 'left').drop("PATENT").withColumnRenamed("POSTATE", "CITING_STATE")

In [16]:
df1.show(5) # This action computes all the transformations and displays result of first 5 rows

+-------+-------+------------+
| CITING|  CITED|CITING_STATE|
+-------+-------+------------+
|3858258|1331793|          CA|
|3858258|1540798|          CA|
|3858527| 924225|        null|
|3858527|2444326|        null|
|3858527|2705120|        null|
+-------+-------+------------+
only showing top 5 rows



Next we do Left join with the intermediate table df1 and df with df1.CITED == df.PATENT and rename POSTATE column to CITED_STATE which is a transformation

In [17]:
df2 = df1.join(df, df1.CITED == df.PATENT, 'left').drop("PATENT").withColumnRenamed("POSTATE", "CITED_STATE")

In [18]:
df2.show(5) # In the next action now the previous transformations on this dataframe are computed and the first 5 rows of resultant
#dataframe are displayed.

+-------+-----+------------+-----------+
| CITING|CITED|CITING_STATE|CITED_STATE|
+-------+-----+------------+-----------+
|4305315| 2366|          MN|       null|
|4192521| 2366|        null|       null|
|4253355| 2366|          MN|       null|
|5580635| 5156|          WI|       null|
|4976561| 5518|        null|       null|
+-------+-----+------------+-----------+
only showing top 5 rows



Filter out NULL entried from CITING_STATE and CITED_STATE columns from dataframe df2 [Transformation]

In [19]:
df3 = df2.filter("CITING_STATE is not Null and CITED_STATE is not Null")

In [20]:
df3.show() # In this action, The filter transformation is now applied and the resultant dataframe with non null entries in both
# CITING_STATE and CITED_STATE are returned.

+-------+-------+------------+-----------+
| CITING|  CITED|CITING_STATE|CITED_STATE|
+-------+-------+------------+-----------+
|4496943|3071753|          NJ|         MN|
|4345315|3071753|          TN|         MN|
|4120573|3071753|          IL|         MN|
|3949375|3071753|          NJ|         MN|
|4271479|3071753|          NY|         MN|
|4280448|3072100|          KS|         IL|
|3861359|3072100|          IL|         IL|
|4138968|3072100|          KS|         IL|
|3894516|3072100|          OK|         IL|
|4572109|3072100|          SC|         IL|
|4396343|3072274|          OH|         WI|
|4955781|3072274|          OR|         WI|
|4907934|3072274|          OR|         WI|
|3917094|3072274|          WI|         WI|
|3908563|3073661|          CT|         IL|
|4615276|3073661|          RI|         IL|
|5926985|3073661|          MI|         IL|
|4546302|3074006|          MN|         CA|
|4720941|3074211|          FL|         MA|
|3947152|3075459|          NY|         OH|
+-------+--

Now weaApply filter tranformation to take subset of dataframe where CITING_STATE == CITED_STATE

In [21]:
df4 = df3.filter("CITING_STATE == CITED_STATE")

In [22]:
df4.show() # This Action Filter transformation is computed and returned dataframe contains records where CITING_STATE == CITED_STATE

+-------+-------+------------+-----------+
| CITING|  CITED|CITING_STATE|CITED_STATE|
+-------+-------+------------+-----------+
|4383847|3177062|          AL|         AL|
|3878730|3610053|          AL|         AL|
|3877317|3610053|          AL|         AL|
|4601891|3733191|          AL|         AL|
|4237106|3733191|          AL|         AL|
|4028087|3733191|          AL|         AL|
|4427432|3969483|          AL|         AL|
|4246248|3969483|          AL|         AL|
|4090893|3974004|          AL|         AL|
|4429634|3974004|          AL|         AL|
|4471988|4415080|          AL|         AL|
|4725766|4546264|          AL|         AL|
|4730154|4546264|          AL|         AL|
|4735382|4546264|          AL|         AL|
|5049030|4547115|          AL|         AL|
|4880701|4698246|          AL|         AL|
|4806399|4698246|          AL|         AL|
|5213858|4698246|          AL|         AL|
|4888222|4698246|          AL|         AL|
|4983431|4698246|          AL|         AL|
+-------+--

Now, we will grouby CITING and perform aggregate function - count

In [23]:
df5 = df4.drop("CITED", "CITING_STATE", "CITED_STATE").groupby("CITING").count() # Transformation

In [24]:
df5.show(10) # In this action it  Computes the earlier transformation and returns result of groupby with count.

+-------+-----+
| CITING|count|
+-------+-----+
|4214280|    2|
|4712293|    5|
|5968543|    8|
|5723969|    4|
|5279657|    2|
|5489298|   16|
|5354420|    1|
|5976783|    5|
|4899423|    1|
|5550181|    5|
+-------+-----+
only showing top 10 rows



Now we perform a left join of patents dataframe with df5 on patents.PATENT == df5.CITING and we drop CITING column.

In [25]:
df6 = patents.join(df5, patents.PATENT == df5.CITING, 'left').drop("CITING") # Transformation

Next fill the NULL values in count column in df6 with 0.

In [26]:
df6 = df6.na.fill({'count': 0}) # Transformation

In the next Transformation we use orderby count column in descending order.

In [27]:
df7 = df6.orderBy('count', ascending=False) 

Next action displays the first 10 records of dataframe sorted in descending order.

In [28]:
df7.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|  125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|  103|
|6008204| 1999|14606|   1998|     US|     CA|  749584| 